In [38]:
import re

location = "Seattle,WA"
print(location)

print(location)

Seattle,WA
seattle,wa


In [39]:
city, state = location.split(",")[0].capitalize(), location.split(",")[1].upper()



print(city)
print(state)

Seattle
WA


In [7]:

df = scrape_historical_sales(location)

Start:2015-01-01 End:2015-04-01 Count:1308
Start:2015-04-02 End:2015-06-30 Count:2327
Start:2015-07-01 End:2015-09-28 Count:2254
Start:2015-09-29 End:2015-12-27 Count:1833
Start:2015-12-28 End:2016-03-26 Count:1352
Start:2016-03-27 End:2016-06-24 Count:2508
Start:2016-06-25 End:2016-09-22 Count:2532
Start:2016-09-23 End:2016-12-21 Count:2134
Start:2016-12-22 End:2017-03-21 Count:1555
Start:2017-03-22 End:2017-06-19 Count:2379
Start:2017-06-20 End:2017-09-17 Count:2521
Start:2017-09-18 End:2017-12-16 Count:2441
Start:2017-12-17 End:2018-03-16 Count:1641
Start:2018-03-17 End:2018-06-14 Count:2373
Start:2018-06-15 End:2018-09-12 Count:2375
Start:2018-09-13 End:2018-12-11 Count:1973
Start:2018-12-12 End:2019-03-11 Count:1549
Start:2019-03-12 End:2019-06-09 Error:'INDUSTRIAL' is not a valid PropertyType
Start:2019-06-10 End:2019-09-07 Count:2763
Start:2019-09-08 End:2019-12-06 Count:2490
Start:2019-12-07 End:2020-03-05 Count:2027
Start:2020-03-06 End:2020-06-03 Count:2277
Start:2020-06-04 E

In [8]:
dataset = PropertyDatasetProcessor(df, city).clean_dataset()

C:\Users\roman.bilets\AppData\Local\Temp\ipykernel_105688\72580958.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['distance_to_downtown'] = dataset.apply(lambda row: self.calc_lat_lon_dist(row['latitude'], row['longitude'], self.downtown_lat, self.downtown_lon), axis=1)
C:\Users\roman.bilets\AppData\Local\Temp\ipykernel_105688\72580958.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['baths'] = dataset.apply(lambda row: self.calc_baths_num(row["full_baths"], row["half_baths"]), axi

In [9]:
model = train_model(dataset)

Model trained with the score: 0.7991304167587265


In [16]:
serialized_model = pickle.dumps(model)

In [11]:
print("Model size: ", len(serialized_model))

Model size:  60522111


{'etag': '"0x8DC6176454EAE69"',
 'last_modified': datetime.datetime(2024, 4, 20, 20, 12, 57, tzinfo=datetime.timezone.utc),
 'content_md5': None,
 'content_crc64': bytearray(b'h\xc5\x12~\xbc\xa5\xaa8'),
 'client_request_id': '611b448f-ff52-11ee-98d6-f0b61e5d13e8',
 'request_id': '0bc97858-501e-0065-465f-932b52000000',
 'version': '2023-11-03',
 'version_id': None,
 'date': datetime.datetime(2024, 4, 20, 20, 12, 57, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}

In [20]:
import os

import base64
import pickle
from azure.storage.blob import BlobServiceClient, BlobClient

# Assuming 'config' is defined elsewhere in your project


# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Create a unique blob name

blob_client = container_client.get_blob_client(blob=blob_name)

# Serialize the model
serialized_model = pickle.dumps(model)

# Function to divide the model into chunks
def get_chunks(data, chunk_size=4 * 1024 * 1024):  # Default to 4MB chunks
    index = 0
    while index < len(data):
        yield data[index:index + chunk_size]
        index += chunk_size

# List to store block ids
block_ids = []

# Upload the chunks
for i, chunk in enumerate(get_chunks(serialized_model)):
    block_id = str(uuid.uuid4()).encode('utf-8')  # Generate a unique block id
    base64_block_id = base64.b64encode(block_id).decode('utf-8')
    blob_client.stage_block(block_id=base64_block_id, data=chunk, length=len(chunk))
    block_ids.append(base64_block_id)

# Commit the blocks to create the final blob
blob_client.commit_block_list([{'id': id} for id in block_ids])

print(f"{blob_name} uploaded successfully.")


HttpResponseError: The specified blob or block content is invalid.
RequestId:fcdc58ae-301e-004c-345d-931526000000
Time:2024-04-20T20:03:12.3019470Z
ErrorCode:InvalidBlobOrBlock
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>InvalidBlobOrBlock</Code><Message>The specified blob or block content is invalid.
RequestId:fcdc58ae-301e-004c-345d-931526000000
Time:2024-04-20T20:03:12.3019470Z</Message></Error>

In [ ]:
# folder_path = os.path.join(os.getcwd(), 'historical_data')

# dataframes = []


# for file_name in os.listdir(folder_path):
#     if file_name.endswith('.csv'):
#         file_path = os.path.join(folder_path, file_name)
#         df = pd.read_csv(file_path)
#         dataframes.append(df)

# print(len(dataframes))
# combined_df = pd.concat(dataframes, ignore_index=True)

In [ ]:
# y_pred = rf_model.predict(X_test)
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print(f'Test RMSE: {rmse}')

# rf_score = rf_model.score(X_test,y_test)
# expl_rf = explained_variance_score(y_pred, y_test)


# models_score =pd.DataFrame({'Model':['Random forest Regression'],
#                             'Score':[rf_score],
#                             'Explained Variance Score':[expl_rf]
#                            })
# models_score.sort_values(by='Score',ascending=False)

----------------------------------------------------------------------------------------------------------------------

In [ ]:
# plt.figure(figsize=(10,6))
# sns.plotting_context('notebook',font_scale=1.2)
# g = sns.pairplot(dataset, hue='beds',height=2)
# g.set(xticklabels=[])

In [ ]:
# sns.jointplot(x='zip_code',y='sold_price',data=dataset,kind='reg',height=4)
# sns.jointplot(x='beds',y='sold_price',data=dataset,kind='reg',height=4)
# sns.jointplot(x='sqft',y='sold_price',data=dataset,kind='reg',height=4)
# sns.jointplot(x='year_built',y='sold_price',data=dataset,kind='reg',height=4)
# sns.jointplot(x='last_sold_date',y='sold_price',data=dataset,kind='reg',height=4)
# sns.jointplot(x='lot_sqft',y='sold_price',data=dataset,kind='reg',height=4)
# sns.jointplot(x='stories',y='sold_price',data=dataset,kind='reg',height=4)
# sns.jointplot(x='hoa_fee',y='sold_price',data=dataset,kind='reg',height=4)
# sns.jointplot(x='parking_garage',y='sold_price',data=dataset,kind='reg',height=4)
# sns.jointplot(x='distance_to_downtown',y='sold_price',data=dataset,kind='reg',height=4)
# sns.jointplot(x='baths',y='sold_price',data=dataset,kind='reg',height=4)

In [ ]:
# plt.figure(figsize=(15,10))
# sns.heatmap(dataset.corr(),annot=True)